In [1]:
import os
import pandas as pd
import numpy as np
import subprocess
from tqdm import tqdm
import sys

In [2]:
sys.path

['/Users/mazeyu/Desktop/CMU/20fall/18797/project',
 '/Users/mazeyu/opt/anaconda3/envs/mlsp/lib/python38.zip',
 '/Users/mazeyu/opt/anaconda3/envs/mlsp/lib/python3.8',
 '/Users/mazeyu/opt/anaconda3/envs/mlsp/lib/python3.8/lib-dynload',
 '',
 '/Users/mazeyu/opt/anaconda3/envs/mlsp/lib/python3.8/site-packages',
 '/Users/mazeyu/opt/anaconda3/envs/mlsp/lib/python3.8/site-packages/IPython/extensions',
 '/Users/mazeyu/.ipython']

In [3]:
DATASET_PATH = "/Users/mazeyu/Desktop/CMU/20fall/18797/project/data"
CONF_PATH = "/Users/mazeyu/Desktop/CMU/20fall/18797/project/opensmile/opensmile/config/is09-13/IS11_speaker_state.conf"
OPENSMILE_PATH = "/Users/mazeyu/Desktop/CMU/20fall/18797/project/opensmile/opensmile/build/progsrc/smilextract/SMILExtract"

DOC_PATH = 'alc_original/DOC/IS2011CHALLENGE'

DATA_PATH = 'alc_original'
TRAIN_TABLE = 'TRAIN.TBL'
D1_TABLE = 'D1.TBL'
D2_TABLE = 'D2.TBL'
TEST_TABLE = 'TESTMAPPING.txt'
SR = 16000

In [4]:
class ALCDataset:
    def __init__(self, path):
        self.dataset_path = path
        self.__load_meta_file()

    def __process_meta(self, meta):
        meta['file_name'] = meta['file_name'].map(lambda x: x[x.find('/') + 1:].lower())
        meta['file_name'] = meta['file_name'].map(lambda x: x[:-8] + 'm' + x[-7:])
        meta['session'] = meta['file_name'].map(lambda x: x[:x.find('/')])
        meta['label'] = meta['user_state'].map(lambda x: 1 if x == 'I' else 0)
        return meta
    
    def __extract_feature(self, wav_input_path, csv_output_path, conf_path):
        subprocess.run([OPENSMILE_PATH, "-C", conf_path, "-I", wav_input_path, "-csvoutput", csv_output_path])
        
        
    def extract_feature(self, split, conf_path):
        split = split.lower()
        assert split in ('train', 'd1', 'd2', 'test')
        meta = getattr(self, f'{split}_meta')
        
        features = []
        for file_name in tqdm(meta['file_name']):
            wav_input_path = os.path.join(self.dataset_path, DATA_PATH, file_name)
            # csv_output_path = f'{wav_input_path.split(".")[0]}.csv'
            csv_output_path = "feature.csv"
            if os.path.exists(csv_output_path):
                os.remove(csv_output_path)
            # print(wav_input_path, csv_output_path)
            self.__extract_feature(wav_input_path, csv_output_path, conf_path)
            feature = pd.read_csv(csv_output_path, delimiter=";").iloc[0, 2:].to_numpy()
            features.append(feature)
            
            
        features = np.stack(features)
        
        np.save(f'{split}_x.npy', features)
            
        return features

    def __load_meta_file(self):
        """Load meta file.

        :return: None
        """
        assert os.path.exists(self.dataset_path)
        doc_folder = os.path.join(self.dataset_path, DOC_PATH)
        print(doc_folder)

        train_meta_path = os.path.join(doc_folder, TRAIN_TABLE)
        self.train_meta = pd.read_csv(train_meta_path, sep='\t', names=['file_name', 'bac', 'user_state'])
        self.train_meta = self.__process_meta(self.train_meta)

        d1_meta_path = os.path.join(doc_folder, D1_TABLE)
        self.d1_meta = pd.read_csv(d1_meta_path, sep='\t', names=['file_name', 'bac', 'user_state'])
        self.d1_meta = self.__process_meta(self.d1_meta)

        d2_meta_path = os.path.join(doc_folder, D2_TABLE)
        self.d2_meta = pd.read_csv(d2_meta_path, sep='\t', names=['file_name', 'bac', 'user_state'])
        self.d2_meta = self.__process_meta(self.d2_meta)

        test_meta_path = os.path.join(doc_folder, TEST_TABLE)
        self.test_meta = pd.read_csv(test_meta_path, sep='\t',
                                     names=['file_name', 'bac', 'user_state', 'test_file_name'])
        self.test_meta = self.test_meta[['file_name', 'bac', 'user_state']]
        self.test_meta = self.__process_meta(self.test_meta)

In [5]:
dataset = ALCDataset(DATASET_PATH)

/Users/mazeyu/Desktop/CMU/20fall/18797/project/data/alc_original/DOC/IS2011CHALLENGE


In [8]:
f_train = dataset.extract_feature("train", CONF_PATH)

100%|██████████| 5400/5400 [29:56<00:00,  3.01it/s]


In [10]:
f_d1 = dataset.extract_feature("d1", CONF_PATH)

100%|██████████| 3960/3960 [23:36<00:00,  2.80it/s]


In [11]:
f_d2 = dataset.extract_feature("d2", CONF_PATH)

100%|██████████| 1500/1500 [09:04<00:00,  2.75it/s]


In [6]:
f_test = dataset.extract_feature("test", CONF_PATH)

100%|██████████| 3000/3000 [14:55<00:00,  3.35it/s]
